# Get LLm Model

In [7]:
from characterai import PyCAI

client = PyCAI('dd7a926b001ecb0100c5022075f51b7cc8893cdd')

char = 'YntB_ZeqRq2l_aVf2gWDCZl4oBttQzDvhj9cXafWcF8'

chat = client.chat.get_chat(char)

participants = chat['participants']

if not participants[0]['is_human']:
    tgt = participants[0]['user']['username']
else:
    tgt = participants[1]['user']['username']

while True:
    message = input('You: ')

    data = client.chat.send_message(
        chat['external_id'], tgt, message
    )

    name = data['src_char']['participant']['name']
    text = data['replies'][0]['text']

    print(f"{name}: {text}")

# Get token: JSON.parse(localStorage.getItem('char_token')).value in console

Character Assistant: Hello! I am here to help you. What can I assist you with today?
Character Assistant: Hello! I am here to help you. What can I assist you with today?


# STT

In [6]:
import speech_recognition as sr

def speech_to_text(microphone, recognizer):

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)

    print("Listening...")
    with microphone as source:
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio)
        print("Recognized Text: ", text)

    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")

    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

    return text

if __name__ == "__main__":

    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    while(True):
        speech_to_text(microphone, recognizer)

Listening...


KeyboardInterrupt: 

# TTS

In [1]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
from playsound import playsound
import torch
import random
import string
import soundfile as sf

device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
# load the processor
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# load the model
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
# load the vocoder, that is the voice encoder
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)
# we load this dataset to get the speaker embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")


In [3]:

# speaker ids from the embeddings dataset
speakers = {
    'awb': 0,     # Scottish male
    'bdl': 1138,  # US male
    'clb': 2271,  # US female
    'jmk': 3403,  # Canadian male
    'ksp': 4535,  # Indian male
    'rms': 5667,  # US male
    'slt': 6799   # US female
}


In [4]:

def save_text_to_speech(text, speaker=None):
    # preprocess text
    inputs = processor(text=text, return_tensors="pt").to(device)
    if speaker is not None:
        # load xvector containing speaker's voice characteristics from a dataset
        speaker_embeddings = torch.tensor(embeddings_dataset[speaker]["xvector"]).unsqueeze(0).to(device)
    else:
        # random vector, meaning a random voice
        speaker_embeddings = torch.randn((1, 512)).to(device)
    # generate speech with the models
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    if speaker is not None:
        # if we have a speaker, we use the speaker's ID in the filename
        output_filename = f"../Cache/audio.mp3"
    else:
        # if we don't have a speaker, we use a random string in the filename
        random_str = ''.join(random.sample(string.ascii_letters+string.digits, k=5))
        output_filename = f"../Cache/audio.mp3"
    # save the generated speech to a file with 16KHz sampling rate
    sf.write(output_filename, speech.cpu().numpy(), samplerate=16000)
    # return the filename for reference
    return output_filename


In [5]:

# generate speech with a US female voice

if __name__ == "__main__" : 
    # TTS Function
    save_text_to_speech(text, speaker=speakers["slt"])
    # Playing Sounds
    playsound("../Cache/audio.mp3")


NameError: name 'text' is not defined

In [6]:
from characterai import PyCAI
import os, time

client = PyCAI('dd7a926b001ecb0100c5022075f51b7cc8893cdd')
 
char = 'YntB_ZeqRq2l_aVf2gWDCZl4oBttQzDvhj9cXafWcF8'

chat = client.chat.get_chat(char)

participants = chat['participants']

if not participants[0]['is_human']:
    tgt = participants[0]['user']['username']
else:
    tgt = participants[1]['user']['username']

while True:
    message = input('You: ')

    data = client.chat.send_message(
        chat['external_id'], tgt, message
    )

    name = data['src_char']['participant']['name']
    text = data['replies'][0]['text']
    # TTS Function
    save_text_to_speech(text, speaker=speakers["slt"])
    # Playing Sounds
    playsound("../Cache/audio.mp3")

# Get token: JSON.parse(localStorage.getItem('char_token')).value in console

LibsndfileError: Error opening '../Cache/audio.mp3': System error.

In [ ]:
if __name__ == "__main__" : 
        # TTS Function
        save_text_to_speech(text, speaker=speakers["slt"])
        print(f"{name}: {text}")
        # Playing Sounds
        time.sleep(1)